# Search Engine Project: Phases 1, 2, and 3

This notebook implements a search engine for the Cranfield dataset across three phases:
- **Phase 1**: Indexes the dataset using PyTerrier.
- **Phase 2 (Query Preproccessing)**: Processes queries with TF-IDF ranking, fixing identical scores and restrictive retrieval.
- **Phase 3 (Query Expansion)**: Adds query expansion with relevance feedback, synonym mapping, and BERT, a Gradio interface, and evaluation.

## Phase 1: Indexing the Cranfield Dataset

Preprocessing the dataset, comparing stemming methods, and creating a PyTerrier index.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer

In [2]:
input_file = "cran.all.1400.csv"
output_file = "cran_preprocessed_modern.csv"

In [3]:
print("=== Loading the Cranfield Dataset ===")
data = pd.read_csv(input_file)
df = pd.DataFrame(data)
print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows of raw data:")
print(df.head())

=== Loading the Cranfield Dataset ===
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398 entries, 0 to 1397
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doc_NO          1398 non-null   int64 
 1   Title           1398 non-null   object
 2   Bib             1330 non-null   object
 3   Text            1398 non-null   object
 4   Processed_Text  1398 non-null   object
dtypes: int64(1), object(4)
memory usage: 54.7+ KB
None

First 5 rows of raw data:
   Doc_NO                                              Title  \
0       1  experimental investigation of the aerodynamics...   
1       2  simple shear flow past a flat plate in an inco...   
2       3  the boundary layer in simple shear flow past a...   
3       4  approximate solutions of the incompressible la...   
4       5  one-dimensional transient heat conduction into...   

                                                 Bib  \
0         

In [4]:
print("\n=== Checking for Missing Values ===")
print("Missing values in 'Title':", df['Title'].isna().sum())
print("Missing values in 'Text':", df['Text'].isna().sum())
print("Total rows before dropping NaN:", len(df))


=== Checking for Missing Values ===
Missing values in 'Title': 0
Missing values in 'Text': 0
Total rows before dropping NaN: 1398


In [5]:
df = df.dropna(subset=['Title'])
print("Total rows after dropping NaN in Title:", len(df))
print("\nFirst 5 rows after dropping NaN:")
print(df.head())

Total rows after dropping NaN in Title: 1398

First 5 rows after dropping NaN:
   Doc_NO                                              Title  \
0       1  experimental investigation of the aerodynamics...   
1       2  simple shear flow past a flat plate in an inco...   
2       3  the boundary layer in simple shear flow past a...   
3       4  approximate solutions of the incompressible la...   
4       5  one-dimensional transient heat conduction into...   

                                                 Bib  \
0                         j. ae. scs. 25, 1958, 324.   
1  department of aeronautical engineering, rensse...   
2  department of mathematics, university of manch...   
3                         j. ae. scs. 22, 1955, 728.   
4                         j. ae. scs. 24, 1957, 924.   

                                                Text  \
0  experimental investigation of the aerodynamics...   
1  simple shear flow past a flat plate in an inco...   
2  the boundary layer in simple

In [6]:
print("\n=== Step 1: Cleaning Titles ===")
cleaned_titles = []
for title in df['Title']:
    title_clean = re.sub(r'[^a-zA-Z\s]', '', str(title))
    title_clean = re.sub(r'\s+', ' ', title_clean).strip()
    cleaned_titles.append(title_clean.lower())
df['Cleaned_Title'] = cleaned_titles
print("Sample of cleaned Titles (first 2 rows):")
print(df[['Doc_NO', 'Cleaned_Title']].head(2))


=== Step 1: Cleaning Titles ===
Sample of cleaned Titles (first 2 rows):
   Doc_NO                                      Cleaned_Title
0       1  experimental investigation of the aerodynamics...
1       2  simple shear flow past a flat plate in an inco...


In [7]:
print("\n=== Step 2: Tokenizing Titles and Vocabulary Analysis ===")
vectorizer = CountVectorizer(
    stop_words="english",
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]+\b'
)
vector = vectorizer.fit_transform(df['Cleaned_Title'])
terms = vectorizer.get_feature_names_out()
print("Total unique terms in Titles:", len(terms))
print("First 20 terms in Title vocabulary:", terms[:20])


=== Step 2: Tokenizing Titles and Vocabulary Analysis ===
Total unique terms in Titles: 1804
First 20 terms in Title vocabulary: ['ablating' 'ablation' 'accelerated' 'accelerating' 'according'
 'accumulation' 'accuracy' 'acoustic' 'acoustical' 'acting' 'action'
 'active' 'adapted' 'addendum' 'addition' 'adiabatic' 'adiabaticwall'
 'adjacent' 'advances' 'advancing']


In [8]:
tokenized_titles = []
for title in df['Cleaned_Title']:
    words = title.split()
    tokenized_titles.append(words)
df['Title_Tokens'] = tokenized_titles
print("\nSample tokenized Titles (first 2 rows):")
print(df[['Doc_NO', 'Title_Tokens']].head(2))


Sample tokenized Titles (first 2 rows):
   Doc_NO                                       Title_Tokens
0       1  [experimental, investigation, of, the, aerodyn...
1       2  [simple, shear, flow, past, a, flat, plate, in...


In [9]:
print("\n=== Step 3: Comparing Stemming Methods ===")
porter = PorterStemmer()
snowball = SnowballStemmer("english")
lancaster = LancasterStemmer()


=== Step 3: Comparing Stemming Methods ===


In [10]:
porter_stemmed = []
snowball_stemmed = []
lancaster_stemmed = []
for word in terms:
    porter_stemmed.append(porter.stem(word))
    snowball_stemmed.append(snowball.stem(word))
    lancaster_stemmed.append(lancaster.stem(word))

In [11]:
print("\nStemming Comparison (First 5 Title Terms):")
print("-" * 60)
print(f"{'Original':<15} | {'Porter':<15} | {'Snowball':<15} | {'Lancaster':<15}")
print("-" * 60)
for i in range(min(5, len(terms))):
    print(f"{terms[i]:<15} | {porter_stemmed[i]:<15} | {snowball_stemmed[i]:<15} | {lancaster_stemmed[i]:<15}")
print("-" * 60)


Stemming Comparison (First 5 Title Terms):
------------------------------------------------------------
Original        | Porter          | Snowball        | Lancaster      
------------------------------------------------------------
ablating        | ablat           | ablat           | abl            
ablation        | ablat           | ablat           | abl            
accelerated     | acceler         | acceler         | accel          
accelerating    | acceler         | acceler         | accel          
according       | accord          | accord          | accord         
------------------------------------------------------------


In [12]:
print("\nApplying Snowball Stemming to Title Tokens...")
stemmed_titles = []
for tokens in df['Title_Tokens']:
    stemmed_words = []
    for word in tokens:
        stemmed_words.append(snowball.stem(word))
    stemmed_titles.append(stemmed_words)
df['Stemmed_Title_Tokens'] = stemmed_titles
print("Sample stemmed Titles (first 2 rows):")
print(df[['Doc_NO', 'Stemmed_Title_Tokens']].head(2))


Applying Snowball Stemming to Title Tokens...
Sample stemmed Titles (first 2 rows):
   Doc_NO                               Stemmed_Title_Tokens
0       1  [experiment, investig, of, the, aerodynam, of,...
1       2  [simpl, shear, flow, past, a, flat, plate, in,...


In [13]:
print("\n=== Step 4: Creating Processed_Text from Titles for Indexing ===")
processed_text = []
for stemmed_tokens in df['Stemmed_Title_Tokens']:
    joined = " ".join(stemmed_tokens)
    processed_text.append(joined)
df['Processed_Text'] = processed_text
print("Sample Processed_Text from Titles (first 2 rows):")
print(df[['Doc_NO', 'Processed_Text']].head(2))


=== Step 4: Creating Processed_Text from Titles for Indexing ===
Sample Processed_Text from Titles (first 2 rows):
   Doc_NO                                     Processed_Text
0       1  experiment investig of the aerodynam of a wing...
1       2  simpl shear flow past a flat plate in an incom...


In [14]:
print("\n=== Step 6: Saving Processed Data ===")
output_df = df[['Doc_NO', 'Title', 'Bib', 'Text', 'Processed_Text']]
output_df.to_csv(output_file, index=False)
print("Saved to:", output_file)
print("Final output (first 5 rows):")
print(output_df.head())


=== Step 6: Saving Processed Data ===
Saved to: cran_preprocessed_modern.csv
Final output (first 5 rows):
   Doc_NO                                              Title  \
0       1  experimental investigation of the aerodynamics...   
1       2  simple shear flow past a flat plate in an inco...   
2       3  the boundary layer in simple shear flow past a...   
3       4  approximate solutions of the incompressible la...   
4       5  one-dimensional transient heat conduction into...   

                                                 Bib  \
0                         j. ae. scs. 25, 1958, 324.   
1  department of aeronautical engineering, rensse...   
2  department of mathematics, university of manch...   
3                         j. ae. scs. 22, 1955, 728.   
4                         j. ae. scs. 24, 1957, 924.   

                                                Text  \
0  experimental investigation of the aerodynamics...   
1  simple shear flow past a flat plate in an inco...   
2  

In [15]:
print("\n=== Step 5: Creative Title Insights ===")
print("Average token count per Title:", round(df['Title_Tokens'].apply(len).mean(), 2))
print("Longest Title (tokens):", df['Title_Tokens'].apply(len).max(), "in Doc_NO:",
      df['Doc_NO'][df['Title_Tokens'].apply(len).idxmax()])
print("Most frequent term in Titles (before stemming):")
word_counts = vector.toarray().sum(axis=0)
top_term_idx = word_counts.argmax()
print(f"'{terms[top_term_idx]}' appears {word_counts[top_term_idx]} times")


=== Step 5: Creative Title Insights ===
Average token count per Title: 11.4
Longest Title (tokens): 40 in Doc_NO: 1082
Most frequent term in Titles (before stemming):
'flow' appears 322 times


In [16]:
!pip install python-terrier

In [17]:
import pyterrier as pt

In [18]:
if not pt.java.started():
    pt.java.init()
    print("Java Virtual Machine started!")

Java Virtual Machine started!


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [19]:
input_file = "/content/cran_preprocessed_modern.csv"

In [20]:
df = pd.read_csv(input_file)
print(df.head())

   Doc_NO                                              Title  \
0       1  experimental investigation of the aerodynamics...   
1       2  simple shear flow past a flat plate in an inco...   
2       3  the boundary layer in simple shear flow past a...   
3       4  approximate solutions of the incompressible la...   
4       5  one-dimensional transient heat conduction into...   

                                                 Bib  \
0                         j. ae. scs. 25, 1958, 324.   
1  department of aeronautical engineering, rensse...   
2  department of mathematics, university of manch...   
3                         j. ae. scs. 22, 1955, 728.   
4                         j. ae. scs. 24, 1957, 924.   

                                                Text  \
0  experimental investigation of the aerodynamics...   
1  simple shear flow past a flat plate in an inco...   
2  the boundary layer in simple shear flow past a...   
3  approximate solutions of the incompressible la...  

In [21]:
df["docno"] = df["Doc_NO"].astype(str)
print("\nSample with docno (first 2 rows):")
print(df[['docno', 'Title', 'Processed_Text']].head(2))


Sample with docno (first 2 rows):
  docno                                              Title  \
0     1  experimental investigation of the aerodynamics...   
1     2  simple shear flow past a flat plate in an inco...   

                                      Processed_Text  
0  experiment investig of the aerodynam of a wing...  
1  simpl shear flow past a flat plate in an incom...  


In [22]:
print("\n=== Step 1: Creating and Indexing the Titles ===")
indexer = pt.DFIndexer("./CranfieldTitleIndex", overwrite=True)
index_ref = indexer.index(df["Processed_Text"], df["docno"])
print("Index location:", index_ref.toString())
print("Indexing complete! Stored at:", index_ref.toString())

<ipython-input-22-4bb2647ecaf0>:2: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./CranfieldTitleIndex", overwrite=True)



=== Step 1: Creating and Indexing the Titles ===
Index location: ./CranfieldTitleIndex/data.properties
Indexing complete! Stored at: ./CranfieldTitleIndex/data.properties


In [23]:
print("\n=== Step 2: Loading the Index ===")
index = pt.IndexFactory.of(index_ref)
print("Index loaded successfully!")


=== Step 2: Loading the Index ===
Index loaded successfully!


In [24]:
lexicon = index.getLexicon()
count = 0
for kv in lexicon:
    if count < 10:
        term = kv.getKey()
        entry = kv.getValue()
        print(f"{term} -> Nt={entry.getNumberOfEntries()} TF={entry.getFrequency()} maxTF={entry.getMaxFrequencyInDocuments()}")
        count = count + 1
    else:
        break

ablat -> Nt=12 TF=12 maxTF=1
accel -> Nt=2 TF=2 maxTF=1
accord -> Nt=1 TF=1 maxTF=1
accumul -> Nt=1 TF=1 maxTF=1
accuraci -> Nt=2 TF=2 maxTF=1
acoust -> Nt=5 TF=5 maxTF=1
act -> Nt=1 TF=1 maxTF=1
action -> Nt=1 TF=1 maxTF=1
activ -> Nt=1 TF=1 maxTF=1
adapt -> Nt=1 TF=1 maxTF=1


In [25]:
print("\n=== Step 5: Setting Up Search Function ===")
def search_term(term):
    stemmer = SnowballStemmer("english")
    term = term.lower()
    stemmed_term = stemmer.stem(term)
    print(f"\nSearching for: '{term}' (stemmed: '{stemmed_term}')")
    try:
        pointer = index.getLexicon()[stemmed_term]
        print(f"Found term '{stemmed_term}' with stats: {pointer.toString()}")
        print("Documents containing the term:")
        postings = index.getInvertedIndex().getPostings(pointer)
        for posting in postings:
            doc_id = posting.getId()
            doc_length = posting.getDocumentLength()
            print(f"- Doc ID: {doc_id} (docno: {df['docno'].iloc[doc_id]}), Length: {doc_length}")
    except KeyError:
        print(f"Term '{stemmed_term}' not found in the index.")


=== Step 5: Setting Up Search Function ===


In [26]:
search_term("information")
search_term("Omar")


Searching for: 'information' (stemmed: 'inform')
Found term 'inform' with stats: term700 Nt=1 TF=1 maxTF=1 @{0 5628 7}
Documents containing the term:
- Doc ID: 439 (docno: 440), Length: 8

Searching for: 'omar' (stemmed: 'omar')
Term 'omar' not found in the index.


## Phase 2: Query Processing with TF-IDF Ranking

This phase processes queries, retrieves documents, and ranks them using TF-IDF. It fixes:
- **Identical scores (1.4142)**: Uses full vocabulary and cosine similarity for varied scores.
- **Restrictive retrieval**: Retrieves documents with *any* query term, improving recall for queries like 'information retrieval'.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [28]:
def preprocess_query(query, stemmer=SnowballStemmer('english')):
    query = query.lower()
    query = re.sub(r'[^a-zA-Z\s]', '', query)
    query = re.sub(r'\s+', ' ', query).strip()
    tokens = query.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

# Test
sample_query = 'Experimental Aerodynamics Wing'
print('Sample query:', sample_query)
print('Preprocessed tokens:', preprocess_query(sample_query))

Sample query: Experimental Aerodynamics Wing
Preprocessed tokens: ['experiment', 'aerodynam', 'wing']


In [29]:
def retrieve_documents(query_tokens, index, df):
    lexicon = index.getLexicon()
    doc_ids = set()

    for token in query_tokens:
        try:
            pointer = lexicon[token]
            postings = index.getInvertedIndex().getPostings(pointer)
            doc_ids.update(posting.getId() for posting in postings)
        except KeyError:
            print(f"Term '{token}' not found in index.")

    if not doc_ids:
        return []

    results = []
    for doc_id in doc_ids:
        docno = df['docno'].iloc[doc_id]
        title = df['Title'].iloc[doc_id]
        processed_text = df['Processed_Text'].iloc[doc_id]
        results.append({
            'doc_id': doc_id,
            'docno': docno,
            'title': title,
            'processed_text': processed_text
        })

    return results

In [30]:
def rank_documents(documents, query_tokens, original_query_tokens=None):
    if not documents:
        return []

    corpus = [doc['processed_text'] for doc in documents]
    query = ' '.join(query_tokens)

    vectorizer = TfidfVectorizer(
        lowercase=False,
        token_pattern=r'(?u)\b\w+\b',
        norm='l2',
        use_idf=True,
        smooth_idf=True
    )
    try:
        tfidf_matrix = vectorizer.fit_transform(corpus)
        query_vector = vectorizer.transform([query])
        scores = cosine_similarity(query_vector, tfidf_matrix).flatten()

        if original_query_tokens:
            original_query = ' '.join(original_query_tokens)
            original_vector = vectorizer.transform([original_query])
            original_scores = cosine_similarity(original_vector, tfidf_matrix).flatten()
            scores = 0.5 * scores + 0.5 * original_scores
    except ValueError as e:
        print(f'TF-IDF calculation failed: {e}')
        scores = [0] * len(documents)

    for i, doc in enumerate(documents):
        doc['tfidf_score'] = scores[i]

    ranked_docs = sorted(documents, key=lambda x: x['tfidf_score'], reverse=True)
    return ranked_docs

In [31]:
def search(query, index, df, top_k=5):
    print(f'\n=== Searching for: "{query}" ===')
    query_tokens = preprocess_query(query)
    print('Query tokens:', query_tokens)

    documents = retrieve_documents(query_tokens, index, df)
    if not documents:
        print('No documents found.')
        return []
    print(f'Found {len(documents)} documents.')

    ranked_docs = rank_documents(documents, query_tokens)

    print(f'Top {min(top_k, len(ranked_docs))} results:')
    for i, doc in enumerate(ranked_docs[:top_k], 1):
        print(f'{i}. Docno: {doc["docno"]}, Title: {doc["title"]}, TF-IDF Score: {doc["tfidf_score"]:.4f}')

    return ranked_docs

# Test searches
search('aerodynamics wing', index, df)
search('information retrieval', index, df)
search('nonexistent term', index, df)


=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 5 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2512
3. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2509
4. Docno: 749, Title: the aerodynamic effects of aspect ratio and sweepback
    on wing flutter ., TF-IDF Score: 0.2467
5. Docno: 1271, Title: theory of supersonic propeller aerodynamics ., TF-IDF Score: 0.2460

=== Searching for: "information retrieval" ===
Query tokens: ['inform', 'retriev']
Term 'retriev' not found in index.
Found 1 documents.
Top 1 results:
1. Docno: 440, Title: compilation of information on the transonic attachment
    of flows at the leading edge of airfoils ., TF-IDF Score: 0.2085

=== Searching for: "nonexistent term" ===
Query to

[]

## Phase 3: Query Expansion, User Interface, and Evaluation

This phase enhances the search engine with query expansion (relevance feedback, synonym mapping, BERT), a Gradio interface, and evaluation with precision@5.

In [32]:
!pip install transformers==4.44.2 torch==2.4.1 gradio==4.44.0
from collections import Counter
from transformers import BertTokenizer, BertModel
import torch
import gradio as gr
import numpy as np

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def expand_query(query, index, df, expansion_type='bert', top_k=3, num_terms=1):
    query_tokens = preprocess_query(query)
    print(f'Original query tokens: {query_tokens}')


    lexicon = index.getLexicon()
    def is_in_index(term):
        return term in lexicon

    synonym_map = {
        'aerodynam': ['flow', 'lift', 'airfoil'],
        'wing': ['airfoil', 'flap', 'swept'],
        'boundari': ['layer', 'shear', 'turbul'],
        'flow': ['stream', 'current'],
        'experiment': ['test', 'trial'],
        'inform': ['data', 'report', 'compil'],
        'retriev': ['search', 'queri', 'fetch']
    }
    synonym_terms = []
    for token in query_tokens:
        if token in synonym_map:
            synonym_terms.extend([syn for syn in synonym_map[token] if syn not in query_tokens and is_in_index(syn)])
    synonym_terms = synonym_terms[:num_terms]
    print(f'Synonym terms: {synonym_terms}')

    stop_words = set(stopwords.words('english'))
    initial_docs = search(query, index, df, top_k=top_k)
    feedback_terms = []
    if initial_docs:
        all_terms = []
        for doc in initial_docs:
            terms = [t for t in doc['processed_text'].split() if t not in stop_words and t not in synonym_terms and is_in_index(t)]
            all_terms.extend(terms)
        term_counts = Counter(all_terms)
        feedback_terms = [term for term, count in term_counts.most_common() if term not in query_tokens and term not in synonym_terms and term not in stop_words and is_in_index(term)][:num_terms]
        print(f'Relevance feedback terms: {feedback_terms}')
    else:
        print('No documents found for initial query.')

    embedding_terms = []
    if expansion_type == 'bert':
        print('Loading BERT...')
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        vocab = [kv.getKey() for kv in lexicon]
        query_inputs = tokenizer(' '.join(query_tokens), return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            query_outputs = model(**query_inputs)
        query_embed = query_outputs.last_hidden_state[:, 0, :].mean(dim=0).numpy()

        inputs = tokenizer(vocab[:100], return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        vocab_embeds = outputs.last_hidden_state[:, 0, :].numpy()

        similarities = cosine_similarity([query_embed], vocab_embeds).flatten()
        top_indices = np.argsort(similarities)[::-1]
        for i in top_indices:
            if similarities[i] > 0.85:
                term = vocab[i]
                if term not in query_tokens and term not in synonym_terms and term not in feedback_terms and term not in stop_words and is_in_index(term):
                    embedding_terms.append(term)
                    break
        print(f'BERT embedding terms: {embedding_terms}')

    expanded_query = list(set(query_tokens + feedback_terms + synonym_terms + embedding_terms))
    return expanded_query

# Test expansion types
test_query = 'aerodynamics wing'
for exp_type in ['synonym', 'bert']:
    print(f'\nTesting {exp_type.upper()} expansion:')
    expanded_tokens = expand_query(test_query, index, df, expansion_type=exp_type)
    print(f'Expanded query: {expanded_tokens}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Testing SYNONYM expansion:
Original query tokens: ['aerodynam', 'wing']
Synonym terms: ['flow']

=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 3 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2512
3. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2509
Relevance feedback terms: ['superson']
Expanded query: ['wing', 'superson', 'aerodynam', 'flow']

Testing BERT expansion:
Original query tokens: ['aerodynam', 'wing']
Synonym terms: ['flow']

=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 3 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerody

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT embedding terms: ['airforc']
Expanded query: ['wing', 'airforc', 'superson', 'flow', 'aerodynam']


In [34]:
def search_with_expansion(query, index, df, expansion_type='bert', top_k=5):
    print(f'\n=== Searching for "{query}" with {expansion_type.upper()} expansion ===')
    original_query_tokens = preprocess_query(query)
    query_tokens = expand_query(query, index, df, expansion_type=expansion_type)
    print(f'Using query tokens: {query_tokens}')

    documents = retrieve_documents(query_tokens, index, df)
    if not documents:
        print('No documents found.')
        return []
    print(f'Found {len(documents)} documents.')

    ranked_docs = rank_documents(documents, query_tokens, original_query_tokens=original_query_tokens)

    print(f'Top {min(top_k, len(ranked_docs))} results:')
    for i, doc in enumerate(ranked_docs[:top_k], 1):
        print(f'{i}. Docno: {doc["docno"]}, Title: {doc["title"]}, TF-IDF Score: {doc["tfidf_score"]:.4f}')

    return ranked_docs

# Test
search_with_expansion('aerodynamics wing', index, df, expansion_type='bert')


=== Searching for "aerodynamics wing" with BERT expansion ===
Original query tokens: ['aerodynam', 'wing']
Synonym terms: ['flow']

=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 3 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2512
3. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2509
Relevance feedback terms: ['superson']
Loading BERT...


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT embedding terms: ['airforc']
Using query tokens: ['wing', 'airforc', 'superson', 'flow', 'aerodynam']
Found 556 documents.
Top 5 results:
1. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2829
2. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2783
3. Docno: 1271, Title: theory of supersonic propeller aerodynamics ., TF-IDF Score: 0.2756
4. Docno: 895, Title: the airforces on the low aspect ratio rectangular wing
    oscillating in sonic flow ., TF-IDF Score: 0.2722
5. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2706


[{'doc_id': 0,
  'docno': '1',
  'title': 'experimental investigation of the aerodynamics of a\r\n      wing in a slipstream .',
  'processed_text': 'experiment investig of the aerodynam of a wing in a slipstream',
  'tfidf_score': np.float64(0.282940595242996)},
 {'doc_id': 1330,
  'docno': '1333',
  'title': 'aerodynamic forces on wings in non-uniform motion .',
  'processed_text': 'aerodynam forc on wing in nonuniform motion',
  'tfidf_score': np.float64(0.2783137973281986)},
 {'doc_id': 1268,
  'docno': '1271',
  'title': 'theory of supersonic propeller aerodynamics .',
  'processed_text': 'theori of superson propel aerodynam',
  'tfidf_score': np.float64(0.27556825200210233)},
 {'doc_id': 893,
  'docno': '895',
  'title': 'the airforces on the low aspect ratio rectangular wing\r\n    oscillating in sonic flow .',
  'processed_text': 'the airforc on the low aspect ratio rectangular wing oscil in sonic flow',
  'tfidf_score': np.float64(0.2722092079409032)},
 {'doc_id': 414,
  'docn

In [35]:
def search_query_interface(query, expansion_type):
    expansion_map = {'Synonym': 'synonym', 'BERT': 'bert'}
    exp_type = expansion_map.get(expansion_type, 'bert')
    results = search_with_expansion(query, index, df, expansion_type=exp_type, top_k=10)
    if not results:
        return {'results': []}
    output = [
        {'docno': doc['docno'], 'title': doc['title'], 'tfidf_score': round(doc['tfidf_score'], 4)}
        for doc in results
    ]
    return {'results': output}

# Test interface
test_result = search_query_interface('aerodynamics wing', 'BERT')
print('Interface test output:', test_result)


=== Searching for "aerodynamics wing" with BERT expansion ===
Original query tokens: ['aerodynam', 'wing']
Synonym terms: ['flow']

=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 3 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2512
3. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2509
Relevance feedback terms: ['superson']
Loading BERT...


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT embedding terms: ['airforc']
Using query tokens: ['wing', 'airforc', 'superson', 'flow', 'aerodynam']
Found 556 documents.
Top 10 results:
1. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2829
2. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2783
3. Docno: 1271, Title: theory of supersonic propeller aerodynamics ., TF-IDF Score: 0.2756
4. Docno: 895, Title: the airforces on the low aspect ratio rectangular wing
    oscillating in sonic flow ., TF-IDF Score: 0.2722
5. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2706
6. Docno: 708, Title: aerodynamic characteristics of two winged reentry vehicles at supersonic
    and hypersonic speeds ., TF-IDF Score: 0.2623
7. Docno: 142, Title: the problem of aerodynamic heating ., TF-IDF Score: 0.2594
8. Docno: 1197, Title: unsteady aerodynamic forces on slender supersonic aircraft
   

In [36]:
gr.Interface(
     fn=search_query_interface,
     inputs=[
         gr.Textbox(label='Enter your query', placeholder='e.g., aerodynamics wing'),
         gr.Dropdown(choices=['Synonym', 'BERT'], label='Expansion Type', value='BERT')
     ],
     outputs=gr.JSON(label='Search Results'),
     title='Cranfield Search Engine',
     description='Search the Cranfield dataset with query expansion.'
 ).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.11/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://f31a2d24b0b6ade3c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [37]:
def evaluate_search_engine(index, df):
    test_cases = {
        'aerodynamics wing': ['1', '140', '141'],
        'boundary layer': ['3', '4', '142'],
        'information retrieval': ['440']
    }
    expansion_types = ['synonym', 'bert']
    metrics = {}

    for exp_type in expansion_types:
        print(f'\n=== Evaluating with {exp_type.upper()} expansion ===')
        metrics[exp_type] = {}
        for query, relevant_docnos in test_cases.items():
            print(f'Query: "{query}" (Expected docnos: {relevant_docnos})')
            results = search_with_expansion(query, index, df, expansion_type=exp_type, top_k=5)
            retrieved_docnos = [doc['docno'] for doc in results[:5]]
            relevant_retrieved = len(set(retrieved_docnos) & set(relevant_docnos))
            precision = relevant_retrieved / 5 if retrieved_docnos else 0
            metrics[exp_type][query] = {'precision@5': precision}
            print(f'Retrieved docnos: {retrieved_docnos}')
            print(f'Retrieved titles: {[doc["title"] for doc in results[:5]]}')
            print(f'Precision@5: {precision:.2f} ({relevant_retrieved}/5 relevant)')
            print('Speed: Fast response for small dataset.')

    return metrics

# Run evaluation
evaluation_results = evaluate_search_engine(index, df)
print('\nFinal Evaluation Results:', evaluation_results)


=== Evaluating with SYNONYM expansion ===
Query: "aerodynamics wing" (Expected docnos: ['1', '140', '141'])

=== Searching for "aerodynamics wing" with SYNONYM expansion ===
Original query tokens: ['aerodynam', 'wing']
Synonym terms: ['flow']

=== Searching for: "aerodynamics wing" ===
Query tokens: ['aerodynam', 'wing']
Found 163 documents.
Top 3 results:
1. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2651
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2512
3. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2509
Relevance feedback terms: ['superson']
Using query tokens: ['wing', 'superson', 'aerodynam', 'flow']
Found 556 documents.
Top 5 results:
1. Docno: 1271, Title: theory of supersonic propeller aerodynamics ., TF-IDF Score: 0.3570
2. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a s

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT embedding terms: ['airforc']
Using query tokens: ['wing', 'airforc', 'superson', 'flow', 'aerodynam']
Found 556 documents.
Top 5 results:
1. Docno: 1, Title: experimental investigation of the aerodynamics of a
      wing in a slipstream ., TF-IDF Score: 0.2829
2. Docno: 1333, Title: aerodynamic forces on wings in non-uniform motion ., TF-IDF Score: 0.2783
3. Docno: 1271, Title: theory of supersonic propeller aerodynamics ., TF-IDF Score: 0.2756
4. Docno: 895, Title: the airforces on the low aspect ratio rectangular wing
    oscillating in sonic flow ., TF-IDF Score: 0.2722
5. Docno: 415, Title: the aerodynamic design of section shapes for swept wings ., TF-IDF Score: 0.2706
Retrieved docnos: ['1', '1333', '1271', '895', '415']
Retrieved titles: ['experimental investigation of the aerodynamics of a\r\n      wing in a slipstream .', 'aerodynamic forces on wings in non-uniform motion .', 'theory of supersonic propeller aerodynamics .', 'the airforces on the low aspect ratio rectangul

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT embedding terms: ['acoust']
Using query tokens: ['shear', 'layer', 'boundari', 'laminar', 'acoust']
Found 252 documents.
Top 5 results:
1. Docno: 1278, Title: transition in a separated laminar boundary layer ., TF-IDF Score: 0.2441
2. Docno: 1365, Title: approximate calculation of the laminar boundary layer ., TF-IDF Score: 0.2436
3. Docno: 155, Title: on the solution of the laminar boundary layer equations ., TF-IDF Score: 0.2303
4. Docno: 459, Title: on the solution of the laminar boundary layer equations ., TF-IDF Score: 0.2303
5. Docno: 1302, Title: the development of the boundary layer in supersonic
    shear flow ., TF-IDF Score: 0.2293
Retrieved docnos: ['1278', '1365', '155', '459', '1302']
Retrieved titles: ['transition in a separated laminar boundary layer .', 'approximate calculation of the laminar boundary layer .', 'on the solution of the laminar boundary layer equations .', 'on the solution of the laminar boundary layer equations .', 'the development of the boundary 

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BERT embedding terms: ['altitud']
Using query tokens: ['compil', 'retriev', 'inform', 'altitud', 'data']
Term 'retriev' not found in index.
Found 16 documents.
Top 5 results:
1. Docno: 440, Title: compilation of information on the transonic attachment
    of flows at the leading edge of airfoils ., TF-IDF Score: 0.3356
2. Docno: 779, Title: calculation and compilation of the unsteady lift functions for a rigid
    wing subjected to sinusoidal gusts and to sinusoidal sinking
    oscillations ., TF-IDF Score: 0.0573
3. Docno: 882, Title: the variation of gust frequency with gust velocity
    and altitude ., TF-IDF Score: 0.0493
4. Docno: 806, Title: ground measurements of airplane shock wave noise at
    mach numbers to 2, and at altitudes of 60,000 feet ., TF-IDF Score: 0.0472
5. Docno: 807, Title: ground measurements of the shock wave noise from
    supersonic bomber airplanes in the altitude range from 30,000
    to 50,000 feet ., TF-IDF Score: 0.0413
Retrieved docnos: ['440', '779', 